In [5]:
import cv2
import numpy as np
import pandas as pd
from scipy.io.wavfile import write
import os


# Mapping Functions
def brightness_to_frequency(brightness):
    return 20 + (brightness / 255) * (20000 - 20)

def rgb_to_frequency(r, g, b):
    return 20 + ((r + g + b) / (3 * 255)) * (20000 - 20)

def rgb_to_amplitude(r, g, b):
    return (r + g + b) / (3 * 255)

# Image Processing and CSV Conversion
def image_to_csv(image_path, csv_path, target_size=(200, 200)):
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError("Image not found. Please check the file path.")
    
    # Resize the image to the target size (200x200) very important to reduce load on system and reduce the computation time otherwise 
    # too much time will be taken to process image with a large number of pixels or high quality images
    image = cv2.resize(image, target_size)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    height, width, _ = image.shape
    
    with open(csv_path, 'w') as file:
        file.write('Row,Column,Red,Green,Blue\n')
        for row in range(height):
            for col in range(width):
                r, g, b = image[row, col]
                file.write(f'{row},{col},{r},{g},{b}\n')
    print(f"Image data saved to {csv_path}")

# Sonification Modes
def sonify_brightness(csv_path, output_audio):
    """based on brightness"""
    pixel_data = pd.read_csv(csv_path)
    sample_rate = 44100
    duration_per_pixel = 0.01 #to reduce the processing time and produce a sound file with shorter duration,though some clarity may be lost
    samples = []
    
    for _, row in pixel_data.iterrows():
        r, g, b = row['Red'], row['Green'], row['Blue']
        brightness = (r + g + b) / 3
        frequency = brightness_to_frequency(brightness)
        amplitude = brightness / 255
        t = np.linspace(0, duration_per_pixel, int(sample_rate * duration_per_pixel), endpoint=False)
        wave = amplitude * np.sin(2 * np.pi * frequency * t)
        samples.extend(wave)
    
    samples = np.array(samples, dtype=np.float32)
    write(output_audio, sample_rate, samples)
    print(f"Audio saved to {output_audio}")

def sonify_color(csv_path, output_audio):
    """based on RGB values"""
    pixel_data = pd.read_csv(csv_path)
    sample_rate = 44100
    duration_per_pixel = 0.01
    samples = []
    
    for _, row in pixel_data.iterrows():
        r, g, b = row['Red'], row['Green'], row['Blue']
        frequency = rgb_to_frequency(r, g, b)
        amplitude = rgb_to_amplitude(r, g, b)
        t = np.linspace(0, duration_per_pixel, int(sample_rate * duration_per_pixel), endpoint=False)
        wave = amplitude * np.sin(2 * np.pi * frequency * t)
        samples.extend(wave)
    
    samples = np.array(samples, dtype=np.float32)
    write(output_audio, sample_rate, samples)
    print(f"Audio saved to {output_audio}")

# Main Interface
def main():
    print("Astronomical Image Sonification Tool")
    image_path = input("Enter image file path: ")

    # Extract the image name without extension
    image_name = os.path.splitext(os.path.basename(image_path))[0]

    csv_path = f"{image_name}_data.csv"
    image_to_csv(image_path, csv_path)
    
    print("Select Sonification Mode:")
    print("1. Brightness-Based Pitch Modulation")
    print("2. Color-Based Sound Effects")
    mode = input("Enter your choice (1 or 2): ")
    
    if mode == '1':
        sonify_brightness(csv_path, f"{image_name}_brightness_audio.wav")
    elif mode == '2':
        sonify_color(csv_path, f"{image_name}_color_audio.wav")
    else:
        print("Invalid choice.")

if __name__ == '__main__':
    main()


Astronomical Image Sonification Tool


Enter image file path:  hires.jpg


Image data saved to hires_data.csv
Select Sonification Mode:
1. Brightness-Based Pitch Modulation
2. Color-Based Sound Effects


Enter your choice (1 or 2):  1


Audio saved to hires_brightness_audio.wav
